Every 15 minutes call api to get last 15 minutes data
reformat it to be like the other dataframe
make image of it
run inference on new image to return up or down for high and low
send that info to trade executer
append new dataframe and labels from last 15 min to old dataframe 

In [ ]:
import torch
import numpy as np
import pandas as pd
import json
import requests

In [ ]:
def minute_scraper():
    url = 'https://api.pro.coinbase.com/products/ETH-USD/candles'
    parameters = {
        'start':'2020-06-09T09:00:40.043459',
        'end':'2020-06-09T09:18:22.021138',
        'granularity': '60'
     }
    r = requests.get(url, parameters)
    return pd.DataFrame(r.json())

eth15 = minute_scraper()
eth15

In [ ]:
days_in_months = {
    '01': 31,
    '02': 28,
    '03': 31,
    '04' :30,
    '05': 31,
    '06' :30,
    '07' :31,
    '08' :31,
    '09' :30,
    '10' :31,
    '11' :30, 
    '12' :31
    }

In [ ]:
year = 2020

url = 'https://api.pro.coinbase.com/products/ETH-USD/candles'
days = 0
for i in days_in_months:
    month = i
    for e in range(1,days_in_months[i]+1):        
        if e < 10:
            day = '0'+str(e)
        else:
            day = str(e)
        start = 0
        end = 3
        count = 0
        for i in range(6):
            if count < 2:
                zerostart = '0'
                zeroend = '0'
            elif count == 2:
                zerostart = '0'
                zeroend = ''
            else:
                zerostart = ''
                zeroend = ''
                
            time.sleep(3)
            parameters = {
                'start':f'{year}-{month}-{day}T{zerostart}{start}:00:00.000000',
                'end':f'{year}-{month}-{day}T{zeroend}{end}:59:00.000000',
                'granularity': '60'
             }
            start += 4
            end += 4
            count += 1
            print(parameters)
            r = requests.get(url, parameters)
            newdf = pd.DataFrame(r.json())
            combined = pd.concat([newdf,eth],ignore_index=True)
            eth = combined
        days +=1
        print(days)


In [ ]:
def Bolinger_Bands(stock_price, window_size, num_of_std):

    rolling_mean = stock_price.rolling(window=window_size).mean()
    rolling_std  = stock_price.rolling(window=window_size).std()
    upper_band = rolling_mean + (rolling_std*num_of_std)
    lower_band = rolling_mean - (rolling_std*num_of_std)

    return rolling_mean, upper_band, lower_band

In [ ]:
def cleaner(ticker):
    ticker['datetime'] = pd.to_datetime(ticker['0'], unit='s')\
               .dt.strftime('%Y-%m-%d %H:%M:%S')
    
    ticker = ticker.drop(['0'], axis=1)

    ticker = ticker.set_index(['datetime'])
    
    ticker = ticker.rename(columns={'1': "Low", '2': "High", '3': "Open", '4': "Close", '5': "Volume"})
    
    rolling_avg_price, upper_band, lower_band = Bolinger_Bands(ticker['Close'], 60, 2)

    ticker['MA'] = rolling_avg_price
    ticker['up_band'] = upper_band
    ticker['down_band'] = lower_band

    ticker = ticker.iloc[::-1]

    #Change order of columns
    cols = ticker.columns.tolist()
    cols = ['Open', 'High', 'Low', 'Close', 'Volume', 'MA', 'up_band', 'down_band']
    ticker = ticker[cols]
    
    
    ticker = ticker.dropna()
    
    return ticker